In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

img_size = 28
n_channels = 1
x_dim = img_size * img_size * n_channels
h_dim = 400
z_dim = 10

print(f'x_dim: {x_dim}, h_dim: {h_dim}, z_dim: {z_dim}')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

batch_size = 128
datadir = '../data'
transform = transforms.Compose([
            transforms.ToTensor()
            ])

train = datasets.MNIST(root=datadir, train=True, transform=transform, download=True)
test = datasets.MNIST(root=datadir, train=False, transform=transform, download=True)

train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

x_dim: 784, h_dim: 400, z_dim: 10
device: cuda


In [2]:
from aistudy.vae import VAE
from tqdm import tqdm
import torch.nn.functional as F
from collections import OrderedDict

vae = VAE(x_dim, h_dim, z_dim).to(device)

# print(vae)
optimizer = torch.optim.Adam(vae.parameters())
n_epochs = 20

for epoch in range(n_epochs):
    # Train phase
    vae.train()
    train_loss = 0
    with tqdm(train_dataloader, unit="batch") as tepoch:
        for i, (x, _) in enumerate(tepoch):
            tepoch.set_description(f"Train phase of epoch {epoch+1}")
            # forward
            x = x.view(-1, x_dim)
            x = x.to(device)
            x_reconst, mu, logvar = vae(x)
            # Compute the reconstruction loss and the regularization term, KL divergence, which is q(x|z) || p(z)
            reconst_loss = F.binary_cross_entropy(x_reconst, x, reduction='sum')
            kl_div = 0.5 * torch.sum(mu.pow(2) + logvar.exp() - logvar - 1)
            loss = reconst_loss + kl_div
            tepoch.set_postfix(OrderedDict({
                'reconst_loss': reconst_loss.item(),
                'kl_div': kl_div.item(),
                'loss': loss.item()
            }))

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
    print(f"Average train loss in epoch {epoch + 1}: {train_loss/len(train_dataloader):.4f}")

    # Test phase
    vae.eval()
    test_loss = 0
    with torch.no_grad(), tqdm(test_dataloader, unit="batch") as tepoch:
        for i, (x, _) in enumerate(tepoch):
            tepoch.set_description(f"Test phase of epoch {epoch+1}")
            # forward
            x = x.view(-1, x_dim)
            x = x.to(device)
            x_reconst, mu, logvar = vae(x)
            # Compute the reconstruction loss and the regularization term, KL divergence, which is q(x|z) || p(z)
            reconst_loss = F.binary_cross_entropy(x_reconst, x, reduction='sum')
            kl_div = 0.5 * torch.sum(mu.pow(2) + logvar.exp() - logvar - 1)
            loss = reconst_loss + kl_div
            tepoch.set_postfix(OrderedDict({
                'reconst_loss': reconst_loss.item(),
                'kl_div': kl_div.item(),
                'loss': loss.item()
            }))
            test_loss += loss.item()
    print(f"Average test loss in epoch {epoch + 1}: {test_loss/len(test_dataloader):.4f}")



Train phase of epoch 1: 100%|██████████| 469/469 [00:02<00:00, 171.74batch/s, reconst_loss=1.69e+4, kl_div=670, loss=1.76e+4]    


Average train loss in epoch 1: 25847.0622


Test phase of epoch 1: 100%|██████████| 79/79 [00:00<00:00, 283.00batch/s, reconst_loss=2.93e+3, kl_div=122, loss=3.05e+3]    


Average test loss in epoch 1: 23120.9199


Train phase of epoch 2: 100%|██████████| 469/469 [00:02<00:00, 217.30batch/s, reconst_loss=1.61e+4, kl_div=915, loss=1.7e+4]     


...

Average test loss in epoch 19: 21317.5509


Train phase of epoch 20: 100%|██████████| 469/469 [00:02<00:00, 221.15batch/s, reconst_loss=1.44e+4, kl_div=1.64e+3, loss=1.61e+4]


Average train loss in epoch 20: 21882.5916


Test phase of epoch 20: 100%|██████████| 79/79 [00:00<00:00, 267.27batch/s, reconst_loss=2.42e+3, kl_div=213, loss=2.63e+3]    

Average test loss in epoch 20: 21322.4300
